In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv
/kaggle/input/catfitmodel/catfit


In [2]:
#!pip install -U feature-engine

In [3]:
import catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.model_selection import train_test_split

In [4]:
#Optuna objective function
def objective(trial):
    params = {
        "iterations": 3000,    # OPtuna example not specify iteration
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostRegressor(**params, silent=True,random_seed=51)
    model.fit(train_pool,verbose=0,eval_set=val_pool)
    y_pred = model.predict_proba(val_pool)
    #rmse = mean_squared_error(y_val, predictions, squared=False)
    return eval_metric(val_pool.get_label(),y_pred[:,1],'RMSE')

In [5]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

In [6]:
df_train['StartYM']=pd.to_datetime(df_train['Policy Start Date']).dt.strftime('%Y-%m')
df_train['StartY']=pd.to_datetime(df_train['Policy Start Date']).dt.strftime('%Y')

In [7]:
df_train['StartYM'].value_counts()

StartYM
2022-05    22166
2023-03    21624
2022-08    21532
2021-07    21404
2022-11    21396
           ...  
2019-12    18581
2024-06    18210
2019-11    17889
2019-08     8868
2024-08     8289
Name: count, Length: 61, dtype: int64

In [8]:
df_train['StartY'].value_counts().reset_index().sort_values(by=['StartY'],ascending=False)

,StartY,count
4,2024,143968
3,2023,239067
0,2022,245671
1,2021,244918
2,2020,242087
5,2019,84289


In [9]:
df_train[df_train['Premium Amount']<120].groupby('StartY')['Premium Amount'].describe()

,count,mean,std,min,25%,50%,75%,max
StartY,,,,,,,,
2019,7231.0,54.466602,32.858719,20.0,28.0,39.0,85.0,119.0
2020,20849.0,57.919181,33.883559,20.0,28.0,45.0,95.0,119.0
2021,21528.0,58.132293,33.804661,20.0,28.0,45.0,95.0,119.0
2022,21401.0,57.777066,33.739692,20.0,28.0,45.0,94.0,119.0
2023,21287.0,58.690046,33.791888,20.0,28.0,47.0,96.0,119.0
2024,12591.0,57.725756,33.505835,20.0,28.0,45.0,93.0,119.0


In [10]:
df_test['Annual Income'].describe()

count    770140.000000
mean      32803.871471
std       32201.063749
min           2.000000
25%        8048.000000
50%       23981.000000
75%       44660.000000
max      149997.000000
Name: Annual Income, dtype: float64

In [11]:
df_train[df_train['Annual Income']<200]['Credit Score'].value_counts()

Credit Score
658.0    26
673.0    26
613.0    25
711.0    25
783.0    24
         ..
525.0     1
355.0     1
348.0     1
440.0     1
405.0     1
Name: count, Length: 539, dtype: int64

In [12]:
df_test[df_test['Annual Income']<200]['Annual Income'].value_counts()

Annual Income
24.0     294
35.0     278
34.0     193
26.0     155
17.0     155
        ... 
30.0       1
12.0       1
160.0      1
150.0      1
123.0      1
Name: count, Length: 184, dtype: int64

In [13]:
#only Age, Anuual Income,Credit Score are number. The rest are category
numeric_columns=df_train.select_dtypes(include=np.number).columns
numeric_columns

Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Premium Amount'],
      dtype='object')

In [14]:
object_columns=df_train.select_dtypes(include=object).columns
object_columns

Index(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Policy Start Date', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type', 'StartYM',
       'StartY'],
      dtype='object')

In [15]:
#transform target
df_train['Label']=np.log1p(df_train['Premium Amount'])
df_train['Label'].describe()

count    1.200000e+06
mean     6.593889e+00
std      1.095825e+00
min      3.044522e+00
25%      6.244167e+00
50%      6.771936e+00
75%      7.319865e+00
max      8.517193e+00
Name: Label, dtype: float64

In [16]:
#Check Na credit score and premium amount
# There a lot of missing score with a lot of premium no different with having creditscor
df_train[df_train['Credit Score'].notna()].groupby('Premium Amount').size().reset_index(name='count').assign(total=lambda x:x['Premium Amount']*x['count']).groupby('count')['total'].sum()

count
1        720797.0
2       1155398.0
3       1264602.0
4       1489848.0
5       2094605.0
          ...    
3009      84252.0
3122      71806.0
3396      67920.0
3444      82656.0
3735      93375.0
Name: total, Length: 917, dtype: float64

In [17]:
#from feature_engine.transformation import YeoJohnsonTransformer

#tf = YeoJohnsonTransformer(variables = ['Annual Income', 'Label'])

#tf.fit(df_train.dropna())
#tf.lambda_dict_

In [18]:
#df_train_tf=tf.transform(df_train.dropna())

In [19]:
df_train[df_train['Premium Amount']<120].loc[:,['Annual Income','Credit Score']].sort_values(ascending=True,by=['Annual Income']).head()

,Annual Income,Credit Score
470076,2.0,332.0
145069,7.0,462.0
120276,11.0,479.0
1187366,14.0,682.0
67382,16.0,513.0


In [20]:
df_train[df_train['Premium Amount']<120]['Credit Score'].describe()

count    91651.000000
mean       614.447764
std        140.831463
min        300.000000
25%        505.000000
50%        623.000000
75%        733.000000
max        849.000000
Name: Credit Score, dtype: float64

In [ ]:
df_train['Credit Score'].describe()

In [ ]:
#Exploratory relation with missing Age and Premium
df_train['group Age']=df_train['Age'].notna().replace({True: 'Age Not NaN', False: 'Age NaN'})
X=df_train_tf['Label']
#sns.relplot(data=df_train,x='Age',y='Premium Amount',hue='group Age',hue_order=['Age Not NaN','Age NaN'],palette={'Age Not NaN':'gray','Age NaN':'red'},kind='scatter')
#sns.plot(x=df_train['Credit Score'].isna(),y=df_train['Premium Amount'],kind='scatter')
sns.displot(data=df_train[df_train['Premium Amount']>=120]['Premium Amount'])
sns.displot(data=df_train[df_train['Premium Amount']<120]['Premium Amount'])
sns.relplot(y=df_train[df_train['Premium Amount']<120]['Credit Score'],x=df_train[df_train['Premium Amount']<120]['Annual Income'],kind='scatter')
plt.title('Distribution Income with premium >=120')
plt.show()

#sns.pairplot(data=df_train, vars=['Credit Score', 'Health Score', 'Premium Amount'], hue='Premium Amount', palette='coolwarm')
#plt.show()


In [21]:
#change fillna from mean to mode to get less effect from outlier
Ana=df_train['Age'].mode()[0]
Aina=df_train['Annual Income'].mode()[0]
Ndna=0
Hsna=df_train['Health Score'].mode()[0]
Pcna=0
Vana=0 #just impute with 0
Csna=df_train['Credit Score'].mode()[0]
Idna=0

In [23]:
df_train['Age'].fillna(Ana,inplace=True)
df_train['Annual Income'].fillna(Aina,inplace=True)
df_train['Number of Dependents'].fillna(Ndna,inplace=True)
df_train['Health Score'].fillna(Hsna,inplace=True)
df_train['Previous Claims'].fillna(Pcna,inplace=True)
df_train['Vehicle Age'].fillna(Vana,inplace=True)
df_train['Credit Score'].fillna(Csna,inplace=True)
df_train['Insurance Duration'].fillna(Idna,inplace=True)


#Ana=df_test['Age'].mean()
#Aina=df_test['Annual Income'].mean()
#Ndna=0
#Hsna=df_test['Health Score'].mean()
#Pcna=0
#Vana=0 #just impute with 0
#Csna=df_test['Credit Score'].mean()
#Idna=0

df_test['Age'].fillna(Ana,inplace=True)
df_test['Annual Income'].fillna(Aina,inplace=True)
df_test['Number of Dependents'].fillna(Ndna,inplace=True)
df_test['Health Score'].fillna(Hsna,inplace=True)
df_test['Previous Claims'].fillna(Pcna,inplace=True)
df_test['Vehicle Age'].fillna(Vana,inplace=True)
df_test['Credit Score'].fillna(Csna,inplace=True)
df_test['Insurance Duration'].fillna(Idna,inplace=True)



/tmp/ipykernel_24/3042408332.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Age'].fillna(Ana,inplace=True)
/tmp/ipykernel_24/3042408332.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [97]:
df_train[df_train['Age'].isna()].index

Index([], dtype='int64')

In [98]:
df_train['Age']=df_train['Age'].fillna(53.0)

In [93]:
df_train['Age'].mode()[0]

53.0

In [24]:
df_train['Occupation'].fillna('Unknown',inplace=True)
df_train['Marital Status'].fillna('Unknown',inplace=True)
df_train['Customer Feedback'].fillna('Unknown',inplace=True)

df_test['Occupation'].fillna('Unknown',inplace=True)
df_test['Marital Status'].fillna('Unknown',inplace=True)
df_test['Customer Feedback'].fillna('Unknown',inplace=True)


/tmp/ipykernel_24/1312047004.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Occupation'].fillna('Unknown',inplace=True)
/tmp/ipykernel_24/1312047004.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [ ]:
#cat_features=numeric_columns.drop(['id','Age','Annual Income','Credit Score','Health Score','Premium Amount']).union(object_columns.drop(['group Age']))
#cat_features.to_list()


In [25]:
cat_features=numeric_columns.drop(['id','Age','Annual Income','Credit Score','Health Score','Premium Amount']).union(object_columns)
cat_features.to_list()


['Customer Feedback',
 'Education Level',
 'Exercise Frequency',
 'Gender',
 'Insurance Duration',
 'Location',
 'Marital Status',
 'Number of Dependents',
 'Occupation',
 'Policy Start Date',
 'Policy Type',
 'Previous Claims',
 'Property Type',
 'Smoking Status',
 'StartY',
 'StartYM',
 'Vehicle Age']

In [26]:
#change cat columns datatype to astype int
df_train[cat_features.intersection(numeric_columns)]=df_train[cat_features.intersection(numeric_columns)].astype('Int64')

#text_features = cat_features.remove('Vehicle Age')

In [27]:
#change cat columns datatype to int astype
df_test[cat_features.intersection(numeric_columns)]=df_test[cat_features.intersection(numeric_columns)].astype('Int64')

#text_features = cat_features.remove('Vehicle Age')

In [20]:
#text_features = cat_features.difference(numeric_columns)
#text_features

Index(['Customer Feedback', 'Education Level', 'Exercise Frequency', 'Gender',
       'Location', 'Marital Status', 'Occupation', 'Policy Start Date',
       'Policy Type', 'Property Type', 'Smoking Status'],
      dtype='object')

In [28]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1200000 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1200000 non-null  float64
 4   Marital Status        1200000 non-null  object 
 5   Number of Dependents  1200000 non-null  Int64  
 6   Education Level       1200000 non-null  object 
 7   Occupation            1200000 non-null  object 
 8   Health Score          1200000 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       1200000 non-null  Int64  
 12  Vehicle Age           1200000 non-null  Int64  
 13  Credit Score          1200000 non-null  float64
 14  Insurance Duration    1200000 non-

In [ ]:
#custom loss function but will not be able to optimization with it so better transform target instead
#def rmsle_loss(y_true, y_pred):
#    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))
#Also use exp to transform the prediction
#predictions = np.expm1(model.predict(X_test))

In [ ]:
#df_train.drop(['id','Annual Income','Customer Feedback','Exercise Frequency','Premium Amount','group Age','Label'],axis=1).columns

In [ ]:
df_train.columns

In [ ]:
#df_train2 = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
#df_train2=df_train2.dropna()


In [ ]:
#nonadata = pd.concat([df_train2['Credit Score'],df_train2['Health Score']],axis=1)
#nonadata.head()

In [ ]:
#nonadata.describe()
#from sklearn.preprocessing import StandardScaler, RobustScaler

#standard_scaler=StandardScaler()
#scalenonadata=standard_scaler.fit_transform(nonadata)

#robust_scaler=RobustScaler()
#scalenonadata=robust_scaler.fit_transform(nonadata)

In [ ]:
#nonadata=pd.DataFrame(scalenonadata,columns=['Credit Score','Health Score'])

In [ ]:
#df_train2['Label'] = np.log1p(df_train2['Premium Amount'])

In [29]:
data = df_train.drop(['id','Premium Amount','Policy Start Date','StartYM','Label'],axis=1) 
cat_features=cat_features.intersection(data.columns)
cat_features=cat_features.difference(['Credit Score','Health Score','Annual Income'])
#text_features=text_features.intersection(data.columns)

In [35]:

cat_features

Index(['Customer Feedback', 'Education Level', 'Exercise Frequency', 'Gender',
       'Insurance Duration', 'Location', 'Marital Status',
       'Number of Dependents', 'Occupation', 'Policy Type', 'Previous Claims',
       'Property Type', 'Smoking Status', 'StartY', 'Vehicle Age'],
      dtype='object')

In [30]:
data.columns

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type', 'StartY'],
      dtype='object')

In [31]:
numeric_columns

Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Premium Amount'],
      dtype='object')

In [38]:
df_train[cat_features]=df_train[cat_features].astype('category')

In [39]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   id                    1200000 non-null  int64   
 1   Age                   1200000 non-null  float64 
 2   Gender                1200000 non-null  category
 3   Annual Income         1200000 non-null  float64 
 4   Marital Status        1200000 non-null  category
 5   Number of Dependents  1200000 non-null  category
 6   Education Level       1200000 non-null  category
 7   Occupation            1200000 non-null  category
 8   Health Score          1200000 non-null  float64 
 9   Location              1200000 non-null  category
 10  Policy Type           1200000 non-null  category
 11  Previous Claims       1200000 non-null  category
 12  Vehicle Age           1200000 non-null  category
 13  Credit Score          1200000 non-null  float64 
 14  Insurance Duration

In [48]:
from catboost import *

X_train,x_val,Y_train,y_val = train_test_split(df_train.drop(['id','StartYM','Premium Amount','Policy Start Date','Label'],axis=1),df_train['Label'],test_size=0.2,random_state=11)
train_pool=Pool(data=X_train,
                label=Y_train,
                cat_features=cat_features.to_list())
val_pool=Pool(data=x_val,
               label=y_val,
               cat_features=cat_features.to_list())



In [49]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=123)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=20)

[I 2024-12-26 19:40:13,596] A new study created in memory with name: no-name-cc14cdcf-4638-47c6-a64c-4895a541e465
[W 2024-12-26 19:53:03,039] Trial 0 failed with parameters: {'learning_rate': 0.02471373418487883, 'depth': 3, 'subsample': 0.26550888088599295, 'colsample_bylevel': 0.5737490306287467, 'min_data_in_leaf': 72} because of the following error: AttributeError("'CatBoostRegressor' object has no attribute 'predict_proba'").
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_24/1364194467.py", line 14, in objective
    y_pred = model.predict_proba(val_pool)
AttributeError: 'CatBoostRegressor' object has no attribute 'predict_proba'
[W 2024-12-26 19:53:03,046] Trial 0 failed with value None.


AttributeError: 'CatBoostRegressor' object has no attribute 'predict_proba'

In [ ]:
#data = nonadata
#label2 = df_train2['Label'].reset_index(drop=True)

In [ ]:
#df_train2['Insurance Duration'].unique()

In [27]:
from sklearn.model_selection import train_test_split 

data = df_train.drop(['id','Premium Amount','group Age','Label'],axis=1) 
label=df_train['Label']
X_train,x_val,Y_train,y_val = train_test_split(data,label,test_size=0.3,random_state=11)

train_data = Pool(data=X_train,
                  label=Y_train,
                  cat_features=cat_features.to_list(),
                  text_features=text_features.to_list(),
                  #text_feature_params={'occurence_lower_bound':1}
                  )

eval_data =  Pool(data=x_val,
                  label=y_val,
                  cat_features=cat_features.to_list(),
                  text_features=text_features.to_list(),)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from catboost import CatBoostRegressor, Pool
from catboost.datasets import msrank

param = {
    'learning_rate': [0.03],
    'depth':[8, 10, 12],
    'l2_leaf_reg': [10,12],
    'iterations':[3000,8000]
}

bestparam = {
    'learning_rate': 0.03,
    'depth':8,
    'l2_leaf_reg':10,
    'iterations':3000,
    'early_stopping_rounds':200,
    'eval_metric':'RMSE',
    'save_snapshot':True,
    'snapshot_file':'catregsnap',
    'snapshot_interval':600,
    'verbose':300,
    'text_features':text_features.to_list(),
   # 'occurrence_lower_bound':1,
    
}





model = CatBoostRegressor(dictionaries=[{'dictionary_id':'Unigram','occurence_lower_bound':'1'},{'dictionary_id':'BiGram','gram_count':'2'},{'dictionary_id':'Word','occurence_lower_bound':'1'}])
model.set_params(**bestparam)
model.fit(X=train_data,eval_set=eval_data,use_best_model=True)
#grid_results = model.grid_search(param,train_data,cv=2,shuffle=False,stratified=True)



In [ ]:
model.save_model('catfit')

In [4]:
from catboost import CatBoostRegressor,Pool

model=CatBoostRegressor()
model.load_model('/kaggle/input/catfitmodel/catfit')

In [37]:
#refit model 
refit = CatBoostRegressor(
    learning_rate=0.02,
    iterations=int(model.tree_count_*1.5),
    dictionaries=[{'dictionary_id':'Unigram','occurence_lower_bound':'1'},{'dictionary_id':'BiGram','gram_count':'2'},{'dictionary_id':'Word','occurence_lower_bound':'1'}]
)

refit.fit(eval_data,verbose=200)

0:	learn: 1.0948654	total: 710ms	remaining: 53m 3s
200:	learn: 1.0556812	total: 2m 6s	remaining: 44m 55s
400:	learn: 1.0541827	total: 3m 35s	remaining: 36m 36s
600:	learn: 1.0532731	total: 5m 1s	remaining: 32m 26s
800:	learn: 1.0513190	total: 6m 44s	remaining: 31m
1000:	learn: 1.0500546	total: 8m 24s	remaining: 29m 14s
1200:	learn: 1.0491191	total: 10m	remaining: 27m 21s
1400:	learn: 1.0482680	total: 11m 35s	remaining: 25m 30s
1600:	learn: 1.0474797	total: 13m 7s	remaining: 23m 39s
1800:	learn: 1.0466781	total: 14m 42s	remaining: 21m 55s
2000:	learn: 1.0459085	total: 16m 16s	remaining: 20m 12s
2200:	learn: 1.0451763	total: 17m 49s	remaining: 18m 30s
2400:	learn: 1.0444737	total: 19m 23s	remaining: 16m 50s
2600:	learn: 1.0437753	total: 20m 55s	remaining: 15m 10s
2800:	learn: 1.0430863	total: 22m 27s	remaining: 13m 30s
3000:	learn: 1.0423941	total: 24m	remaining: 11m 52s
3200:	learn: 1.0417230	total: 25m 33s	remaining: 10m 15s
3400:	learn: 1.0410884	total: 27m 4s	remaining: 8m 38s
3600:	

In [6]:
df_test.isna().any()

id                      False
Age                      True
Gender                  False
Annual Income            True
Marital Status           True
Number of Dependents     True
Education Level         False
Occupation               True
Health Score             True
Location                False
Policy Type             False
Previous Claims          True
Vehicle Age              True
Credit Score             True
Insurance Duration       True
Policy Start Date       False
Customer Feedback        True
Smoking Status          False
Exercise Frequency      False
Property Type           False
dtype: bool

In [ ]:
#Trying Keras but it also produce 1.0+ sth 

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split

# Assuming `X` (features) and `y` (log1p transformed target) are defined
X_train, X_val, y_train, y_val = train_test_split(nonadata, label2, test_size=0.2, random_state=42)

# Neural Network Model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Training the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
                    ])

# Evaluate the model
loss, rmse = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation RMSE: {rmse}")

# Predicting and transforming the target back
y_pred = model.predict(X_val)
y_pred_original_scale = np.expm1(y_pred)


In [38]:
test=df_test.drop(['id'],axis=1)
test.columns
#predicts = model.predict(df_test.drop('id',axis=1))

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type'],
      dtype='object')

In [ ]:
df_train.drop(['id','Premium Amount','group Age','Label'],axis=1).columns

In [ ]:
from sklearn.model_selection import train_test_split

data = df_train.drop(['id','Premium Amount','group Age','Label'],axis=1) 
label=df_train['Label']

X_train,x_val,Y_train,y_val = train_test_split(data,label,test_size=0.3,random_state=11)

predicts = model.predict(x_val)

In [ ]:
test.head()

In [ ]:
x_val.head()

In [ ]:
cat_features.to_list()

In [ ]:
grid_results['params']

In [ ]:
grid_results['cv_results']['train-RMSE-mean']

In [ ]:
predicts = model.predict(test)

In [39]:
#refit model prediction
predicts  = refit.predict(test)

In [40]:
predicts_original_scale = np.expm1(predicts)

In [41]:
#round int
temp=pd.concat([df_test['id'],pd.DataFrame(np.rint(predicts_original_scale),columns=['Premium Amount'])],axis=1).reset_index(drop=True)

In [43]:
#not round int
temp2=pd.concat([df_test['id'],pd.DataFrame(predicts_original_scale.astype(int),columns=['Premium Amount'])],axis=1).reset_index(drop=True)

In [42]:
temp.to_csv('submission_round.csv',index=False)

In [44]:
temp2.to_csv('submission_not_round.csv',index=False)